In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM articles \
                   WHERE datePublished<'2021-01-01' \
                   ORDER BY datePublished", sql, index_col="id", parse_dates=["datePublished"])

In [ ]:
df.set_index("datePublished").resample('D').count()[["url"]].plot()

In [ ]:
df["datePublished"] = pd.to_datetime(df["datePublished"], utc=True)

In [ ]:
df.set_index("datePublished").resample('D').count()[["url"]].plot()

In [ ]:
df.set_index("datePublished").resample('W').count()[["url"]].plot()

In [ ]:
df.set_index("datePublished").resample('M')[["url"]].count().plot()

In [ ]:
top_authors = df.groupby("author").count()[["url"]].sort_values("url")[-20:]
top_authors.plot.barh()

In [ ]:
df["month"] = df["datePublished"].dt.strftime("%m")

In [ ]:
am = df.groupby(["author", "month"], as_index=False).agg({"url": "count"})
am = am[am["url"]>10]
am

In [ ]:
!pip install seaborn

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(20,20))
sns.heatmap(am.pivot(index="author", columns="month", values="url").fillna(0))

In [ ]:
ak = pd.DataFrame([[author, k, 1] for (author, keywords) in zip(df["author"], df["keywords"]) 
                         for k in str(keywords).split(", ")], columns=["author", "keyword", "count"]).dropna()

In [ ]:
akg = ak.groupby(["author", "keyword"], as_index=False).count()

In [ ]:
akg_top = akg[akg["count"]>50].pivot(index="author", columns="keyword", values="count").fillna(0)

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(akg_top)